# Trylib example

## RF

In [ ]:
param = {'n_estimators': 100,
             'criterion' : 'gini',
             'max_depth':None,
             'max_features' : 'auto',
             'min_samples_leaf' : 1,
         
             'n_jobs':-1,
}

res = trylib(X,Y,'rf',param,one = False,skf_seed = 12, skf = 10, X_test= X_test).res

# ET

In [ ]:
param = {'n_estimators': 100,
         'criterion' : 'gini',
         'max_depth':None,
         'max_features' : 'auto',
         'min_samples_leaf' : 1,
         
         'n_jobs':-1,
}

res = trylib(X,Y,'et',param,one = False,skf_seed = 12, skf = 10, X_test= X_test).res

## XGBoost

In [ ]:
%%capture --no-stdout --no-display
param = {'num_round': 200,
         
             'seed' : 2441,
             'max_depth':4,
             'gamma': 0,
             'eta':0.02,
             'min_child_weight':1,
             'silent':1, 

             'objective':'binary:logistic',
             'scale_pos_weight':3,

             'subsample' : 0.95,
             'colsample_bytree' : 0.2,
             #'base_score':Y.mean()
           }

# Objectives:

#  -- 'objective':'binary:logistic',
#  -- 'objective':'multi:softprob',
#  -- 'objective':'rank:pairwise',
#  -- 'objective':'reg:linear',

# eval_metrics:

#  -- 'eval_metric':'mlogloss',
#  -- 'eval_metric':'auc',

# num_class:

#  -- 'num_class':9

res = trylib(X,Y,'xg',param,one = False,skf_seed = 12, skf = 10, X_test= X_test).res

## Linear SVC

In [ ]:
param = {'C':1.0,
         'penalty':'l2',
         'loss':'squared_hinge',
         'dual':True,
         'multi_class':'ovr',
         'class_weight': None,
         'random_state':329,
        }

res = trylib(X,Y,'lsvc',param,one = False,skf_seed = 12, skf = 10, X_test= X_test).res

## KNN

In [ ]:
param = {'n_neighbors':5,
         'weights':'uniform',
         'metric':'minkowski',
        }

# weights:

# -- 'weights':'uniform'
# -- 'weights':'distance'
# [callable]

# metrics:

# -- 'metric':'euclidean'
# -- 'metric':'cityblock'
# -- 'metric':'canberra'
# -- 'metric':'braycurtis'
# cosine ? 

res = trylib(X,Y,'knn',param,one = False,skf_seed = 12, skf = 10, X_test= X_test).res

# Holdout validation

In [ ]:
%%capture --no-stdout --no-display
from sklearn.cross_validation import train_test_split

for i in range(4):
    x_train, x_ho, y_train, y_ho = train_test_split(X, Y, 
                                                test_size=0.25, 
                                                random_state=i)
#     x_train  = x_train.values
#     x_hohout = x_hohout.values
#     y_train  = y_train.values
#     y_ho     = y_ho.values
    
    param = { }
    
    
    res = trylib_r(x_train,y_train,'rf',param,one = True,skf = 5,X_test=x_ho,skf_seed = 12).res

    print 'cv ',res['loss']
    print 'ho ',RMSE(y_ho,res['pr_test'])    
    
    #print 'cv classes ', res['accuracy']
    #print 'ho classes ', accuracy_score(y_ho,np.argmax(res['pr_test'],axis=1)) 

# 2 level

### 1

In [ ]:
%%capture --no-stdout --no-display

# X = 
# Y = 
# X_test =

# ============ 
y_metas = []
y_tests = []

xg_param = {'num_round': 50,
             'seed' : 21,
             'max_depth':3,
             'gamma': 0.01,
             'eta':0.3,
             'min_child_weight':1,
             'silent':1, 
             'objective':'multi:softprob',
             'num_round':100,
             'num_class' : #9,
             'subsample' : 0.8,
             'colsample_bytree' : 0.2}

clfs = [
             ('xg', xg_param),
             ('lsvc', {'C' : 100}),
             ('knn', {'n_neighbors' : 10 , 'metric': 'canberra'}),
             ('knn', {'n_neighbors' : 1 , 'metric': 'canberra'}),
             ('nusvc',{'nu' : 0.007}),
             ('rf',{'max_features' : 0.01,'n_estimators' : 4000}),
             ('rf',{'max_features' : 0.3,'n_estimators' : 4000}),
             ('et',{'max_features' : 0.1,'n_estimators' : 4000}),
    ]

for c in clfs:
    print ' ==== ', c[0], ' ====='
        
    res = trylib(X,Y,c[0],c[1],skf =10, X_test = X_test).res
    print " - %s: log_loss: %f, acc: %f" %(c[0], res['loss'],res['accuracy'])
    y_metas.append(res['pr'])
    y_tests.append(res['pr_test'])

y_meta = res['y']

### 2

In [ ]:
%%capture --no-stdout --no-display

xg_param = {
         'seed' : 21,
         'max_depth':3,
         'gamma': 0.01,
         'eta':0.1,
         'silent':1, 
         'objective':'multi:softprob',
         'num_round':200,
         'num_class' : 9,
         'subsample' : 0.8,
         'colsample_bytree' : 0.1}
r = None

metas = np.hstack(y_metas)
test_metas = np.hstack(y_tests)

y_out = []
for i in range(10):
    xg_param['seed'] = (i+34)*15
    
    res = trylib(metas,y_meta,'xg',xg_param,skf = 10,one = False, X_test= test_metas).res
    y_out.append(res['pr_test'])
    
    if r is None:
        r = res['pr']
    else:
        r += res['pr']
    print "%d %f" % (i, log_loss(y_val,r/(i+1)))

pr_test_final = np.mean(y_out,axis = 0)
pr_test_final.shape
    
#res2 = trylib(np.concatenate([d.values[res['val_index'],:],np.hstack(prs)],axis = 1),y_val,'et',{'max_features' : 0.1,'n_estimators' : 20000, 'compute_importances': True},skf = 10,one = False).res

#prs.append(res['pr_val'])
#y_val = res['y_val']

# CV template

In [ ]:
skf = KFold(Y.shape[0], n_folds=10, shuffle=True, random_state=11)

for it, (train_index, test_index) in enumerate(skf):
    X_train, X_val = X[train_index], X[test_index]
    y_train, y_val = Y[train_index], Y[test_index]